# Callbacks
This notebook proposes an extension to the Problem solver notebook.
It introduces the notion of callbacks to handle options in a modular way.

## What is a callback?
A callback is an object that will execute specific actions that will be called at specific times.

Let us explain this concept with the help of an example. Suppose you want to implement an algorithm with 2 main parts. You want to execute code _not_ related to the algorithm itself in between. To achieve this without altering the algorithm conciseness, define the following generic Callback object:

In [15]:
class Callback:
    """
    This is a generic Callback object that defines 4 methods to handle modular options.
    All callbacks should inherit from this one.
    """
    def before_part_1(self, *args, **kwargs):
        pass
    
    def after_part_1(self, *args, **kwargs):
        pass

    def before_part_2(self, *args, **kwargs):
        pass

    def after_part_2(self, *args, **kwargs):
        pass

To give an example of a use case, and to help us keep track of what is happening, define the Progression callback by inheriting from Callback: 

In [25]:
class BasicProgression(Callback):
    """
    This callback follows progression of the algorithm by printing at every steps.
    """
    def before_part_1(self, *args, **kwargs):
        print('Doing something before part 1.')
    
    def after_part_1(self, *args, **kwargs):
        print('Doing something after part 1.')

    def before_part_2(self, *args, **kwargs):
        print('Doing something before part 2.')
        
    def after_part_2(self, *args, **kwargs):
        print('Doing something after part 2.')

Then, inside the `ProblemSolver` class examined previously, you would change the execution of the `solve` method to

In [26]:
class ProblemSolver:
    def solve(self, data_for_the_algo, callbacks_list):
        
        things_callbacks_need = (self,)
        
        # Before part 1
        for callback in callbacks_list: callback.before_part_1(things_callbacks_need)
        
        # Part 1
        self.solve_part_1(data_for_the_algo)
        
        # After part 1
        for callback in callbacks_list: callback.after_part_1(things_callbacks_need)
        
        # Maybe some line of codes
        
        # Before part 2
        for callback in callbacks_list: callback.before_part_2(things_callbacks_need)
        
        # Part 2
        self.solve_part_2(data_for_the_algo)
        
        # After part 2
        for callback in callbacks_list: callback.after_part_2(things_callbacks_need)
        
        return self
    
    def solve_part_1(self, data): pass
    
    def solve_part_2(self, data): pass

In [29]:
callbacks_list = [BasicProgression()]
problem_solver = ProblemSolver()
problem_solver.solve(1, callbacks_list)

Doing something before part 1.
Doing something after part 1.
Doing something before part 2.
Doing something after part 2.


This might seem like we added many lines of code to achieve very little more than what we could have done with some prints. However, there are many advantages:
- Easy to modify: You know exactly where the text you want to print is; you don't have to find somewhere in the algorithm.
- Easy to extend: You can now pass optional arguments to the constructor of the callback instead of to the `solve` method. For example, you can tune the level of verbosity, or you can print in a file instead of the terminal.

But mostly,
- The `solve` method do exactly that, and nothing else.

Let us look at some useful example.

## The Progression Callback
We take a look at a more advanced example of the Progression Callback.

In [19]:
class Progression(Callback):
    """
    This callback follows progression of the algorithm by printing at every steps.
    
    The verbose level are 0, 1 or 2.
    """
    def __init__(self, verbose_level=1):
        self.verbose_level = verbose_level
    
    def before_part_1(self, *args, **kwargs):
        print('Starting algorithm.')
    
    def after_part_1(self, *args, **kwargs):
        if self.verbose_level >= 1:
            print('Part 1 completed.')
        
    def before_part_2(self, *args, **kwargs):
        if self.verbose_level >= 2:
            print('We talk about part 2 of algorithm only if verbose is set high.')
        
    def after_part_2(self, *args, **kwargs):
        print('Finished algorithm')